In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
np.random.seed(0)
data = pd.read_csv("Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("Data\\cle\\Rand_valid.csv", sep=',')

In [3]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [4]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [5]:
from fit9_nn import fit, comb_error , mae_error

In [6]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
       
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = torch.tanh(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [12]:
n_epochs = 2000
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
sig2 = 2 * 950000
PATH = 'Data\\checkpoints\\mytraining2.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [14]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [15]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [16]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining2.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, val_error = mae_error,
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 0.9879197279612223, Validation Loss: 289.87939453125
Epoch 2: Training Loss: 0.9858558376630148, Validation Loss: 289.8719787597656
Epoch 3: Training Loss: 0.9838199416796366, Validation Loss: 289.8641662597656
Epoch 4: Training Loss: 0.981773684422175, Validation Loss: 289.8560485839844
Epoch 5: Training Loss: 0.9798199633757273, Validation Loss: 289.84735107421875
Epoch 6: Training Loss: 0.9778711795806885, Validation Loss: 289.8380126953125
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 0.9759739140669504, Validation Loss: 289.8279113769531
Epoch 8: Training Loss: 0.9740656912326813, Validation Loss: 289.81689453125
Epoch 9: Training Loss: 0.9722487231095632, Validation Loss: 289.8045349121094
Epoch 10: Training Loss: 0.9704353312651316, Validation Loss: 289.7908935546875
Epoch 11: Training Loss: 0.9686428407828013, Validation Loss: 289.77557373046875
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9668829540411631, Validation Loss: 289.7581

Epoch 97: Training Loss: 0.8879375060399374, Validation Loss: 285.9381103515625
Epoch 98: Training Loss: 0.8873961766560873, Validation Loss: 285.9225158691406
Epoch 99: Training Loss: 0.8868912855784098, Validation Loss: 285.9071960449219
Epoch 100: Training Loss: 0.8863641520341238, Validation Loss: 285.891845703125
Epoch 101: Training Loss: 0.885838141043981, Validation Loss: 285.87646484375
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.8853214283784231, Validation Loss: 285.86114501953125
Epoch 103: Training Loss: 0.8848137060801188, Validation Loss: 285.8458557128906
Epoch 104: Training Loss: 0.8842934668064117, Validation Loss: 285.8307189941406
Epoch 105: Training Loss: 0.8837860425313314, Validation Loss: 285.8154296875
Epoch 106: Training Loss: 0.8832882444063822, Validation Loss: 285.80010986328125
New Checkpoint Saved into PATH
Epoch 107: Training Loss: 0.8827811082204183, Validation Loss: 285.784912109375
Epoch 108: Training Loss: 0.8822843531767527, Validation

Epoch 193: Training Loss: 0.856767882903417, Validation Loss: 284.3636474609375
Epoch 194: Training Loss: 0.8566568891207377, Validation Loss: 284.3475036621094
Epoch 195: Training Loss: 0.8565385043621063, Validation Loss: 284.3301086425781
Epoch 196: Training Loss: 0.8564289609591166, Validation Loss: 284.3136901855469
New Checkpoint Saved into PATH
Epoch 197: Training Loss: 0.8563226858774821, Validation Loss: 284.296142578125
Epoch 198: Training Loss: 0.8562221924463908, Validation Loss: 284.2789306640625
Epoch 199: Training Loss: 0.856110413869222, Validation Loss: 284.26226806640625
Epoch 200: Training Loss: 0.8560129702091217, Validation Loss: 284.24615478515625
Epoch 201: Training Loss: 0.8559120496114095, Validation Loss: 284.2290954589844
New Checkpoint Saved into PATH
Epoch 202: Training Loss: 0.8558093110720316, Validation Loss: 284.2121276855469
Epoch 203: Training Loss: 0.8557222485542297, Validation Loss: 284.1962890625
Epoch 204: Training Loss: 0.8556217749913534, Valid

Epoch 289: Training Loss: 0.8505690594514211, Validation Loss: 282.7804260253906
Epoch 290: Training Loss: 0.8505299190680186, Validation Loss: 282.762939453125
Epoch 291: Training Loss: 0.8504850765069326, Validation Loss: 282.74688720703125
New Checkpoint Saved into PATH
Epoch 292: Training Loss: 0.8504420022169749, Validation Loss: 282.7300109863281
Epoch 293: Training Loss: 0.8503993153572083, Validation Loss: 282.7137451171875
Epoch 294: Training Loss: 0.8503584365049998, Validation Loss: 282.69696044921875
Epoch 295: Training Loss: 0.8503147562344869, Validation Loss: 282.68023681640625
Epoch 296: Training Loss: 0.8502714335918427, Validation Loss: 282.662841796875
New Checkpoint Saved into PATH
Epoch 297: Training Loss: 0.8502349158128103, Validation Loss: 282.6458435058594
Epoch 298: Training Loss: 0.8501884937286377, Validation Loss: 282.6282653808594
Epoch 299: Training Loss: 0.8501478234926859, Validation Loss: 282.61181640625
Epoch 300: Training Loss: 0.8501036763191223, Va

Epoch 385: Training Loss: 0.8469423254330953, Validation Loss: 281.0608825683594
Epoch 386: Training Loss: 0.846897174914678, Validation Loss: 281.04205322265625
New Checkpoint Saved into PATH
Epoch 387: Training Loss: 0.8468641142050425, Validation Loss: 281.0232849121094
Epoch 388: Training Loss: 0.8468289772669474, Validation Loss: 281.00433349609375
Epoch 389: Training Loss: 0.846795380115509, Validation Loss: 280.9841003417969
Epoch 390: Training Loss: 0.8467599153518677, Validation Loss: 280.9639587402344
Epoch 391: Training Loss: 0.8467232584953308, Validation Loss: 280.9442138671875
New Checkpoint Saved into PATH
Epoch 392: Training Loss: 0.8466858565807343, Validation Loss: 280.92413330078125
Epoch 393: Training Loss: 0.8466572264830271, Validation Loss: 280.9040222167969
Epoch 394: Training Loss: 0.8466170032819113, Validation Loss: 280.88507080078125
Epoch 395: Training Loss: 0.8465821345647176, Validation Loss: 280.8665771484375
Epoch 396: Training Loss: 0.8465447525183359,

...

Epoch 1566: Training Loss: 0.7971189518769582, Validation Loss: 311.8334655761719
Epoch 1567: Training Loss: 0.7970779240131378, Validation Loss: 311.8620910644531
Epoch 1568: Training Loss: 0.7970462044080099, Validation Loss: 312.0550842285156
Epoch 1569: Training Loss: 0.7970070242881775, Validation Loss: 312.2440185546875
Epoch 1570: Training Loss: 0.7969734271367391, Validation Loss: 312.5648193359375
Epoch 1571: Training Loss: 0.7969377636909485, Validation Loss: 312.2515563964844
Epoch 1572: Training Loss: 0.7968978782494863, Validation Loss: 312.0538024902344
Epoch 1573: Training Loss: 0.7968632678190867, Validation Loss: 312.0443115234375
Epoch 1574: Training Loss: 0.796825552980105, Validation Loss: 312.0393981933594
Epoch 1575: Training Loss: 0.7967944939931234, Validation Loss: 312.26275634765625
Epoch 1576: Training Loss: 0.7967595060666403, Validation Loss: 312.45037841796875
Epoch 1577: Training Loss: 0.7967218061288198, Validation Loss: 312.48687744140625
Epoch 1578: Tr

Epoch 1666: Training Loss: 0.7938839991887411, Validation Loss: 313.8865051269531
Epoch 1667: Training Loss: 0.7938616971174876, Validation Loss: 314.08160400390625
Epoch 1668: Training Loss: 0.7938296596209208, Validation Loss: 313.9331970214844
Epoch 1669: Training Loss: 0.7938012977441152, Validation Loss: 313.83953857421875
Epoch 1670: Training Loss: 0.7937788466612498, Validation Loss: 314.2119445800781
Epoch 1671: Training Loss: 0.7937505046526591, Validation Loss: 314.4622802734375
Epoch 1672: Training Loss: 0.7937181989351908, Validation Loss: 314.021240234375
Epoch 1673: Training Loss: 0.7936882177988688, Validation Loss: 314.04620361328125
Epoch 1674: Training Loss: 0.7936690449714661, Validation Loss: 314.3767395019531
Epoch 1675: Training Loss: 0.793640673160553, Validation Loss: 313.93798828125
Epoch 1676: Training Loss: 0.7936041156450907, Validation Loss: 314.1924743652344
Epoch 1677: Training Loss: 0.7935839891433716, Validation Loss: 314.482666015625
Epoch 1678: Traini

Epoch 1768: Training Loss: 0.7912656366825104, Validation Loss: 314.85272216796875
Epoch 1769: Training Loss: 0.7912418444951376, Validation Loss: 314.8385009765625
Epoch 1770: Training Loss: 0.7912156383196512, Validation Loss: 314.80206298828125
Epoch 1771: Training Loss: 0.7911960681279501, Validation Loss: 315.4674072265625
Epoch 1772: Training Loss: 0.7911719381809235, Validation Loss: 315.8216552734375
Epoch 1773: Training Loss: 0.7911412914594015, Validation Loss: 315.5308837890625
Epoch 1774: Training Loss: 0.7911231617132822, Validation Loss: 315.63385009765625
Epoch 1775: Training Loss: 0.79108726978302, Validation Loss: 315.3951110839844
Epoch 1776: Training Loss: 0.7910755475362142, Validation Loss: 314.5599365234375
Epoch 1777: Training Loss: 0.7910410563151041, Validation Loss: 314.453857421875
Epoch 1778: Training Loss: 0.7910200854142507, Validation Loss: 314.1831970214844
Epoch 1779: Training Loss: 0.7909971376260122, Validation Loss: 314.0060119628906
Epoch 1780: Trai

Epoch 1870: Training Loss: 0.7888804872830709, Validation Loss: 314.7105407714844
Epoch 1871: Training Loss: 0.7888622879981995, Validation Loss: 314.3669128417969
Epoch 1872: Training Loss: 0.7888345122337341, Validation Loss: 313.9946594238281
Epoch 1873: Training Loss: 0.788809061050415, Validation Loss: 314.0823669433594
Epoch 1874: Training Loss: 0.7888156125942866, Validation Loss: 312.9708251953125
Epoch 1875: Training Loss: 0.7887759109338125, Validation Loss: 312.3323059082031
Epoch 1876: Training Loss: 0.7887577414512634, Validation Loss: 313.0829772949219
Epoch 1877: Training Loss: 0.7887300550937653, Validation Loss: 313.2419128417969
Epoch 1878: Training Loss: 0.7887072861194611, Validation Loss: 313.4667663574219
Epoch 1879: Training Loss: 0.7886722087860107, Validation Loss: 313.71490478515625
Epoch 1880: Training Loss: 0.7886755069096884, Validation Loss: 314.81280517578125
Epoch 1881: Training Loss: 0.7886272172133127, Validation Loss: 314.9566345214844
Epoch 1882: Tra

Epoch 1970: Training Loss: 0.7866367399692535, Validation Loss: 313.1695861816406
Epoch 1971: Training Loss: 0.7866195738315582, Validation Loss: 313.0927734375
Epoch 1972: Training Loss: 0.7865942815939585, Validation Loss: 313.1340026855469
Epoch 1973: Training Loss: 0.7865676979223887, Validation Loss: 312.815673828125
Epoch 1974: Training Loss: 0.7865476906299591, Validation Loss: 312.8880310058594
Epoch 1975: Training Loss: 0.7865239580472311, Validation Loss: 312.93231201171875
Epoch 1976: Training Loss: 0.7865068217118582, Validation Loss: 312.4076843261719
Epoch 1977: Training Loss: 0.7864834070205688, Validation Loss: 312.6217041015625
Epoch 1978: Training Loss: 0.7864589989185333, Validation Loss: 312.8387145996094
Epoch 1979: Training Loss: 0.7864399154980978, Validation Loss: 313.096435546875
Epoch 1980: Training Loss: 0.7864186465740204, Validation Loss: 313.4482727050781
Epoch 1981: Training Loss: 0.7863909204800924, Validation Loss: 313.1976318359375
Epoch 1982: Training